#### This Notebook is the sample Demo setup for working with DakkoAI Client Library


In [ ]:
# Import dakko library
from dakkoai import (
    DakkoAIClient,
    DakkoRegistrationModel,
    DakkoSearch,
)

# Import supporting DataScience lib
import matplotlib.pyplot as plt
from datetime import datetime
import matplotlib.dates as mdates
from collections import Counter
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from functools import reduce
import pandas as pd

# Create dakko_client
dakko_client = DakkoAIClient()

# Register new crypto Data Scientist
user_data = DakkoRegistrationModel(
    email="user@example.com", first_name="John", last_name="Doe", user_name="johndoe"
)

status = dakko_client.register(user_data)
print(f"Register User: {status}")
print(dakko_client.client_credentials.model_dump())

# Get access credentials for further connection to Dakko
status = dakko_client.get_access_token()

print(f"Getting access token: {status}")


In [ ]:
# Fire a query on clean data tables and get raw JSON data
SEARCH_ADDRESS = "0xAD9eF19e289dcbC9AB27b83D2dF53CDEFF60f02D".lower()

dakko_sql = DakkoSearch(
    query=f"""
    
    WITH 
        Constants AS (
            SELECT '{SEARCH_ADDRESS}' AS contract, 
            18 AS dec0, 
            18 AS dec1
        ), 
        PreparedData AS (
            SELECT 
                DATE_TRUNC('hour', timestamp(`timestamp`)) AS ts, 
                pow(sqrtPriceX96 / pow(2, 96), 2) * pow(10, dec0 - dec1) AS price, 
                abs(amount0) / pow(10, dec0) AS amount0, 
                liquidity / pow(10, 12) AS liquidity_e12 
            FROM dakko_poc.eth_demo.token_swaps_raw, Constants
            WHERE address = lower(contract)
        ) 
        
        SELECT 
            ts, 
            FIRST(price) AS open, 
            MAX(price) AS high, 
            MIN(price) AS low, 
            LAST(price) AS close, 
            SUM(amount0) AS raw_volume, 
            SUM(amount0 * price) AS volume, 
            SUM(liquidity_e12) AS liquidity_e12 
        FROM PreparedData 
        WHERE ts IS NOT NULL 
        GROUP BY ts 
        ORDER BY ts
    
    """
)

# Plot the OHLCV sticks using Dataframe results from DakkoAI
temp_df = dakko_client.get_query_results(dakko_sql).dataframe


In [ ]:

# let's draw the OHLCV analysis
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Candlestick(
        x=temp_df.index,  # date values
        open=temp_df["open"],
        high=temp_df["high"],
        low=temp_df["low"],
        close=temp_df["close"],
    ),
    secondary_y=True,
)

fig.add_trace(
    go.Bar(
        x=temp_df.index,
        y=temp_df["volume"],
        marker={
            "color": "rgba(128,128,128,0.4)",
        },
    ),
    secondary_y=False,
)
fig.layout.yaxis2.showgrid = False

# change figsize
fig.update_layout(
    # autosize=False,
    # width=1000,
    height=800,
)

fig.show()